In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

In [3]:
datadf

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,num_employees,num_researchers,primary_industry,types_res_used,future_res,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,research_educ,research_educ_cat,research_educ_desc,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,how_pick_events,best_event,events_attend_recent,ideal_conference_size,ideal_conference_size_cat_orig,ideal_conference_size_cat,ideal_structure,ideal_structure_cat,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables,other_conference_types,ideal_topics,ideal_attendees,recommendations,persona_id
resp_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,3,2,0,3,3,3,3,2,1,4,0.0,2.0,maritime,both qualitative and quantitative research,both qualitative and quantitative research,5,5,5,5,5,5,5,5,5,5,"sort of, psychology background.",1.0,NaN,0,1,1,2,3,1,3,3,2,3,3,4,4,3,3,4,2,3,1,3,1,3,4,NaN,NaN,"CHI, INTERACT",depending on the professional values,7.0,7.0,"any, depending on the values",5.0,0,0,0,4,2,2,2,3,2,2,2,NaN,research-practice connections,NaN,research-practice connections,2
288,Academic,4,1.0,1,2,1,3,3,3,3,3,0,3,4.0,3.0,Higher education,Multiple,Multiple,5,5,5,5,5,5,5,5,2,5,"Yes, I was taught how to conduct research",1.0,Research methods and statistics training in bo...,2,3,2,2,2,2,3,3,2,3,2,2,2,3,3,4,3,4,2,3,3,2,4,"Content, cost, timing (can I get away at this ...","Dagstuhl workshop - small, intense, intimate g...",Hawaii international conference on Education,I would attend any of these,6.0,6.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,3,3,3,3,3,1,1,2,NaN,NaN,NaN,Don’t create schedule conflicts with other rel...,2
294,Academic,4,1.0,3,3,1,3,1,3,3,3,1,3,4.0,3.0,University,"User research / Design research\n\nInterviews,...",Online FG,4,4,3,3,4,4,4,4,3,3,"Yes, I was taught how to conduct research",1.0,BA Sociology\nMA LIS\nPhD. LIS\n\nonline cours...,2,3,2,2,2,3,4,3,3,3,3,3,3,3,3,4,4,3,4,3,3,4,3,- time for travel (having two small children]\...,Courses at Copenhagen University (deep knowle...,"Research Tuesdays (Czech), UX/Design Conferen...",It depends - I really enjoyed small conference...,7.0,7.0,It depends,5.0,0,0,0,2,2,2,2,2,2,2,2,NaN,NaN,NaN,Looking forward! \n,2
281,Academic,4,1.0,3,4,0,2,3,2,0,1,0,0,2.0,1.0,Ed Tech,"surveys, interviews, focus groups, secondary d...","surveys, interviews, focus groups, secondary d...",1,1,0,1,1,1,0,1,0,0,"Yes, I was taught how to conduct research",1.0,"I have an Applied Anthropology, MA. I designed...",1,2,0,1,2,3,4,3,2,4,2,3,3,2,3,3,3,1,1,1,2,3,4,My professional development budget allows me t...,A workshop at the Society for Applied Anthropo...,A local education implementation conference.,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,2,3,2,NaN,"Research design, innovative methods, communica...",NaN,NaN,2
280,Academic,4,1.0,3,3,0,1,1,3,2,1,0,0,NaN,NaN,education,NaN,NaN,4,4,0,2,2,3,2,3,0,0,"No, I was not taught how to conduct research",0.0,NaN,1,1,1,1,2,3,3,3,1,2,1,2,1,3,1,4,2,3,1,2,2,4,4,NaN,NaN,NaN,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,1,2,2,You should consider the combination of confere...,NaN,

In [4]:
datadf.describe()

,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,2.856749,2.913223,0.402204,2.600551,1.982094,2.931129,1.845730,2.924242,0.776860,1.860882,3.283747,3.323691,0.960055,2.633609,3.084022,3.139118,1.862259,2.906336,1.326446,1.632231,2.074380,2.659780,1.801653,1.451791,1.663912,2.982094,3.500000,3.263085,2.079890,3.053719,2.202479,2.250689,2.360882,2.236915,2.871901,3.652893,2.776860,2.209366,1.998623,2.028926,1.903581,2.944904,3.336088,0.625344,0.334711,0.061983,2.706612,2.754821,2.341598,2.988981,2.173554,1.480716,2.121212,2.431129
std,1.025941,0.939210,0.720686,1.166285,1.134273,0.950177,1.236637,1.056235,0.914143,1.486931,1.571974,1.553165,1.608530,1.702955,1.694565,1.589165,1.684913,1.711475,1.654099,1.674476,0.791318,0.571707,0.836609,0.784165,0.761119,0.960436,0.711967,0.834391,1.213355,0.983242,1.150254,1.143723,1.101378,1.157435,1.071954,0.590183,0.926135,1.168658,1.107341,1.158115,1.263411,1.019666,0.971309,0.484368,0.472215,0.241292,1.124344,0.977229,1.106273,1.109154,1.219458,1.239149,1.179908,1.211472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000
50%,3.000000,3.000000,0.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,1.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,3.000000
75%,4.000000,3.000000,1.00000

In [5]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 78 columns):
job_title                         726 non-null category
job_taxo                          726 non-null category
job_id                            723 non-null category
job_conduct_res                   726 non-null int64
job_analyze_res                   726 non-null int64
job_buy_res_report                726 non-null int64
job_manage_res_proj               726 non-null int64
job_observe_res                   726 non-null int64
job_plan_res                      726 non-null int64
job_teach_res                     726 non-null int64
job_advocate_res                  726 non-null int64
job_hire_res_vendor               726 non-null int64
job_lead_res_team                 726 non-null int64
num_employees                     723 non-null category
num_researchers                   722 non-null category
primary_industry                  726 non-null category
types_res_used               

## 5. What is your company or organization's primary industry?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

## 6. What types of research do you currently use to make decisions?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

## 7. What types of research are you considering in the future?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

## 10. Describe your educational background with research

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

## 14. How do you decide which events to attend?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

## 22. If attending a conference about research, who might you be excited to see there?

In [37]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [38]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

(<471x925 sparse matrix of type '<class 'numpy.int64'>'
 	with 4580 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [39]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [40]:
lda22.fit(ideal_att_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [41]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.122573  0.065619       1        1  46.076154
2      0.007800 -0.120126       2        1  28.072781
1     -0.130373  0.054507       3        1  25.851065, topic_info=    Category        Freq                  Term       Total  loglift  logprob
668  Default  179.000000              research  179.000000  30.0000  30.0000
557  Default  188.000000                people  188.000000  29.0000  29.0000
514  Default   18.000000                   new   18.000000  28.0000  28.0000
708  Default   15.000000            sam ladner   15.000000  27.0000  27.0000
406  Default   15.000000                ladner   15.000000  26.0000  26.0000
707  Default   15.000000                   sam   15.000000  25.0000  25.0000
775  Default   13.000000                 spool   13.000000  24.0000  24.0000
922  Default   16.000000                 young   16.000000  23.0000  23.0000
380  Default   12.000000           jared spool   12.000000  22.0000  22.0000
195  Default   26.000000                 doing   26.000000  21.0000  21.0000
348  Default   12.000000                  indi   12.000000  20.0000  20.0000
349  Default   12.000000            indi young   12.000000  19.0000  19.0000
836  Default   11.000000                 topic   11.000000  18.0000  18.0000
379  Default   13.000000                 jared   13.000000  17.0000  17.0000
786  Default   22.000000                 steve   22.000000  16.0000  16.0000
4    Default   24.000000              academic   24.000000  15.0000  15.0000
353  Default   41.000000              industry   41.000000  14.0000  14.0000
602  Default   18.000000              portigal   18.000000  13.0000  13.0000
788  Default   17.000000        steve portigal   17.000000  12.0000  12.0000
532  Default    8.000000                norman    8.000000  11.0000  11.0000
308  Default   15.000000                  hall   15.000000  10.0000  10.0000
303  Default   12.000000                 group   12.000000   9.0000   9.0000
853  Default   12.000000                  type   12.000000   8.0000   8.0000
190  Default   13.000000             diversity   13.000000   7.0000   7.0000
811  Default   20.000000                  team   20.000000   6.0000   6.0000
422  Default    8.000000          leader field    8.000000   5.0000   5.0000
226  Default   14.000000                 erika   14.000000   4.0000   4.0000
158  Default    9.000000                   day    9.000000   3.0000   3.0000
547  Default   11.000000                  orgs   11.000000   2.0000   2.0000
889  Default   26.000000                  want   26.000000   1.0000   1.0000
..       ...         ...                   ...         ...      ...      ...
288   Topic3    3.527255               goodwin    4.112702   1.1993  -5.8859
397   Topic3    3.527255           kim goodwin    4.112702   1.1993  -5.8859
693   Topic3    3.526336  researcher different    4.112350   1.1991  -5.8862
735   Topic3    3.522396               sharing    4.113478   1.1977  -5.8873
732   Topic3    3.511522                   set    4.113507   1.1946  -5.8904
188   Topic3    3.503869         diverse group    4.114081   1.1923  -5.8925
499   Topic3    4.188803                 mixed    4.932974   1.1893  -5.7140
832   Topic3    6.814088                 tomer    8.213368   1.1660  -5.2274
158   Topic3    7.339972                   day    9.089237   1.1391  -5.1531
736   Topic3    5.935992                sharon    7.394905   1.1331  -5.3654
303   Topic3    9.038417                 group   12.323398   1.0428  -4.9449
824   Topic3    7.718688               thought   12.431304   0.8762  -5.1028
785   Topic3    5.540375               startup    8.224825   0.9577  -5.4344
786   Topic3   10.494819                 steve   22.213930   0.6030  -4.7955
439   Topic3    4.749443                 leisa    6.578592   1.0270  -5.5884
692   Topic3   19.493240            researcher   82.210862  -0.0864 

In [42]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [43]:
nlp.find_top_documents_per_topic(lda_W22, datadf.ideal_attendees.dropna(), 3)

Top 3 Documents for Topic 0: 

Document 1
i don't have specific name or organization that i would recommend i do think that finding more seasoned people in the research industry with a lot of experience are extremely valuable i feel like we're at a moment in time with research where everybody want to start a research practice in larger company but they're just aren't enough seasoned researcher out there to sort of lead these thing we have a bunch of more junior people trying to fill these roll without necessarily the background or the experience to do so in a thoughtful and effective way

Document 2
i would like to hear from and network with other organizational leader i want to meet people i can learn from no pr or salestype presentation no product pushing although i'd like to learn about tool that people are using i am much more interested in strategy and how to manage operation for large team also speaking a someone on the consulting side it would be nice to hear from leader in that

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [27]:
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.basic_clean)
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.lemmatize)

In [28]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), ngram=(1,3), max_df=.5)
other_conference_types_matrix, other_conference_types_vector

(<258x295 sparse matrix of type '<class 'numpy.int64'>'
 	with 1136 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [29]:
lda20 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [30]:
lda20.fit(other_conference_types_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [31]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.068645  0.103219       1        1  39.503990
2     -0.135698  0.000811       2        1  33.594916
1      0.067052 -0.104029       3        1  26.901094, topic_info=    Category       Freq              Term      Total  loglift  logprob
159  Default  15.000000              nope  15.000000  30.0000  30.0000
216  Default  34.000000           session  34.000000  29.0000  29.0000
256  Default  18.000000             think  18.000000  28.0000  28.0000
289  Default  21.000000          workshop  21.000000  27.0000  27.0000
179  Default   8.000000            poster   8.000000  26.0000  26.0000
22   Default   6.000000              case   6.000000  25.0000  25.0000
208  Default   8.000000        researcher   8.000000  24.0000  24.0000
204  Default  18.000000          research  18.000000  23.0000  23.0000
108  Default   6.000000         important   6.000000  22.0000  22.0000
58   Default   7.000000        discussion   7.000000  21.0000  21.0000
23   Default   5.000000        case study   5.000000  20.0000  20.0000
284  Default  13.000000               way  13.000000  19.0000  19.0000
241  Default   6.000000             study   6.000000  18.0000  18.0000
282  Default   7.000000                wa   7.000000  17.0000  17.0000
180  Default   5.000000    poster session   5.000000  16.0000  16.0000
110  Default   5.000000          industry   5.000000  15.0000  15.0000
286  Default   9.000000              work   9.000000  14.0000  14.0000
83   Default   6.000000            format   6.000000  13.0000  13.0000
155  Default   5.000000        networking   5.000000  12.0000  12.0000
246  Default  14.000000              talk  14.000000  11.0000  11.0000
166  Default   7.000000             panel   7.000000  10.0000  10.0000
264  Default   8.000000             track   8.000000   9.0000   9.0000
145  Default   5.000000              meet   5.000000   8.0000   8.0000
33   Default  31.000000        conference  31.000000   7.0000   7.0000
288  Default   9.000000           working   9.000000   6.0000   6.0000
268  Default   5.000000      unconference   5.000000   5.0000   5.0000
54   Default   5.000000            design   5.000000   4.0000   4.0000
259  Default   8.000000              time   8.000000   3.0000   3.0000
52   Default   4.000000              demo   4.000000   2.0000   2.0000
167  Default   4.000000  panel discussion   4.000000   1.0000   1.0000
..       ...        ...               ...        ...      ...      ...
239   Topic3   1.785075             story   2.335349   1.0443  -5.2312
203   Topic3   1.784944            remote   2.335358   1.0442  -5.2313
148   Topic3   1.784641         mentoring   2.335380   1.0440  -5.2315
44    Topic3   1.784455           covered   2.335398   1.0439  -5.2316
123   Topic3   1.779792             kucha   2.335677   1.0412  -5.2342
171   Topic3   1.779792       pecha kucha   2.335677   1.0412  -5.2342
155   Topic3   4.089924        networking   5.465294   1.0231  -4.4021
268   Topic3   3.302688      unconference   5.494358   0.8040  -4.6159
28    Topic3   2.935188              come   4.703703   0.8414  -4.7339
286   Topic3   4.845099              work   9.569301   0.6324  -4.2327
262   Topic3   4.461284             topic   8.705411   0.6445  -4.3152
157   Topic3   3.883251               new   7.135094   0.7047  -4.4540
132   Topic3   2.551702             level   3.935216   0.8798  -4.8739
289   Topic3   7.211347          workshop  21.040072   0.2422  -3.8350
70    Topic3   5.627474             event  14.439736   0.3707  -4.0830
264   Topic3   4.088536             track   8.803818   0.5460  -4.4025
166   Topic3   3.632360             panel   7.914413   0.5342  -4.5208
120   Topic3   3.978060              just   9.570008   0.4352  -4.4299
139   Topic3   3.072969              love   6.349535   0.5873  -4.6880
74    Topic3   2.579250            expert   4.726498   0.7073  -4.

In [25]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [26]:
nlp.find_top_documents_per_topic(lda_W20, datadf.other_conference_types.dropna(), 3)

Top 3 Documents for Topic 0: 

Document 1
i find most workshop at conference to be chaotic and mundane and i come to conference usually to escape work workshops usually make you  do work  i'd rather experience something interactive and engaging and fun that isn't really work

Document 2
i am there to learn efficiently not socialize so the thing you didn't mention is the availability of course note at least the speakers' slides that i can study before and after the talks i have also been at conference that have a knowledgeable discussant associated with each 1hour talk the discussant ha studied the work before and asks pointed question about pro and cons relationship to other techniques point of failure etc when the speaker know this is coming you are le likely to get just a canned talk and you really get to the meat of the issues

Document 3
to make a conference really be incredible making something that they take away is critical cocreation is a good way to crystalize learning and hav

## 15 What was the best professional learning experience you've ever had?  What made it great?

In [44]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [45]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(datadf.best_event.dropna(), ngram=(1,3), max_df=.5)
best_event_matrix, best_event_vector

(<594x1631 sparse matrix of type '<class 'numpy.int64'>'
 	with 9198 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [46]:
lda15 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [47]:
lda15.fit(best_event_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [48]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.027621 -0.088450       1        1  49.951605
2     -0.070925  0.065849       2        1  28.468267
1      0.098546  0.022601       3        1  21.580128, topic_info=     Category        Freq         Term       Total  loglift  logprob
1592  Default  133.000000     workshop  133.000000  30.0000  30.0000
1438  Default   98.000000           ux   98.000000  29.0000  29.0000
1378  Default   39.000000         time   39.000000  28.0000  28.0000
1069  Default   11.000000      product   11.000000  27.0000  27.0000
48    Default   17.000000     activity   17.000000  26.0000  26.0000
303   Default   36.000000       course   36.000000  25.0000  25.0000
1044  Default   21.000000    practical   21.000000  24.0000  24.0000
376   Default   22.000000    different   22.000000  23.0000  23.0000
1167  Default   23.000000   researcher   23.000000  22.0000  22.0000
136   Default   41.000000         best   41.000000  21.0000  21.0000
577   Default  116.000000        great  116.000000  20.0000  20.0000
829   Default   48.000000          lot   48.000000  19.0000  19.0000
929   Default   43.000000          new   43.000000  18.0000  18.0000
1472  Default    9.000000  ve attended    9.000000  17.0000  17.0000
1511  Default   26.000000     wa great   26.000000  16.0000  16.0000
774   Default   75.000000     learning   75.000000  15.0000  15.0000
329   Default   58.000000          day   58.000000  14.0000  14.0000
1457  Default   12.000000         uxpa   12.000000  13.0000  13.0000
996   Default    7.000000  participate    7.000000  12.0000  12.0000
611   Default   22.000000         hand   22.000000  11.0000  11.0000
924   Default   28.000000   networking   28.000000  10.0000  10.0000
1007  Default   78.000000       people   78.000000   9.0000   9.0000
597   Default   31.000000        group   31.000000   8.0000   8.0000
1587  Default   44.000000         work   44.000000   7.0000   7.0000
434   Default   23.000000         epic   23.000000   6.0000   6.0000
118   Default   24.000000     attendee   24.000000   5.0000   5.0000
1282  Default    6.000000         step    6.000000   4.0000   4.0000
51    Default    8.000000     actually    8.000000   3.0000   3.0000
1471  Default   36.000000           ve   36.000000   2.0000   2.0000
1350  Default    9.000000     teaching    9.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
1167   Topic3   11.818324   researcher   23.758702   0.8351  -5.1974
14     Topic3    4.131389         2017    5.826175   1.1897  -6.2485
122    Topic3    4.123713     audience    5.827558   1.1876  -6.2503
136    Topic3   17.170104         best   41.595109   0.6486  -4.8239
202    Topic3    4.116358       choose    5.828222   1.1857  -6.2521
886    Topic3    6.400449       method   10.882756   1.0026  -5.8107
577    Topic3   31.892927        great  116.571156   0.2373  -4.2047
1511   Topic3   11.064091     wa great   26.028280   0.6779  -5.2634
1007   Topic3   21.491249       people   78.190305   0.2419  -4.5994
1483   Topic3   50.726872           wa  317.500873  -0.3006  -3.7406
329    Topic3   16.989496          day   58.769131   0.2924  -4.8345
114    Topic3   15.161315     attended   48.664860   0.3672  -4.9483
1152   Topic3   21.062590     research   86.102399   0.1254  -4.6196
597    Topic3   11.653877        group   31.635650   0.5348  -5.2115
446    Topic3   18.896969        event   74.478662   0.1619  -4.7281
1592   Topic3   25.834894     workshop  133.475779  -0.1088  -4.4154
1333   Topic3   18.486002         talk   77.385178   0.1016  -4.7501
434    Topic3    9.680169         epic   23.893873   0.6299  -5.3970
829    Topic3   13.845837          lot   48.320033   0.2835  -5.0391
1471   Topic3   11.738081           ve   36.467215   0.3998  -5.2043
118    Topic3    9.513467     attendee   24.433302   0.5902  -5.4144
559    Topic3   13.193152     

In [50]:
lda_W15 = lda15.transform(best_event_matrix)

In [51]:
nlp.find_top_documents_per_topic(lda_W15, datadf.best_event.dropna(), 3)

Top 3 Documents for Topic 0: 

Document 1
i hate to say it because it wa so long ago but grad school it wa a small and intimate group of u learning together i think our whole class wa under 50 people hand on training and experience challenge with real client ie we built game for mtv amongst others i walked away with a huge amount of confidence because my skill were tested in the classroom and i felt like i could do it with any company i worked with i'll admit that many year later that confidence ha waned while i think that may just be burn out a conference that can bring together a real client and a bunch of designer to work through something like a service design project maybe start with a cut off time and those that have signed up get to start with call before the conference a someone doe research on the ground then they can join together in person and go through the research and create a customer journey just some idea

Document 2
so many i dont have one favorite i love the euro ia 